<a href="https://colab.research.google.com/github/giseletoledo/assistenteia-docs/blob/main/Assistente_busca_documentacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 📘 Assistente UIKit UIButton com FAISS + LLM local
# Etapa 1: Instalar dependências necessárias
!pip install -U beautifulsoup4 markdownify transformers sentence-transformers faiss-cpu langchain-core langchain-community langchain-huggingface

In [6]:
# Etapa 2: Extrair texto da documentação local HTML (UIButton)
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from pathlib import Path
from langchain.schema import Document

In [7]:
# Função para extrair e converter conteúdo relevante de um HTML salvo
def extrair_markdown_uibutton_html(caminho_html: str) -> list[Document]:
    caminho = Path(caminho_html)
    docs = []

    if not caminho.exists():
        raise FileNotFoundError("Arquivo não encontrado")

    with open(caminho, encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

        # Conteúdo útil geralmente está no <main> ou <article> em documentação técnica
        bloco = soup.find("main") or soup.find("article") or soup.body

        if not bloco:
            raise ValueError("Não foi possível encontrar bloco principal (main/article/body)")

        texto_md = md(str(bloco))

        # Divide por título ou tamanho máximo (~1200 tokens)
        partes = texto_md.split("\n## ")
        for i, parte in enumerate(partes):
            parte_formatada = "## " + parte if i > 0 else parte
            if len(parte_formatada.strip()) > 20:
                docs.append(Document(page_content=parte_formatada.strip(), metadata={"origem": caminho.name}))

    return docs

In [ ]:
# Extrair documentos a partir do HTML salvo localmente
docs = extrair_markdown_uibutton_html("docs/UIButton_Apple_Developer_Documentation.html")
print(f"📄 Documentos extraídos: {len(docs)}")
print(docs[0].page_content[:1000])

In [ ]:
# Etapa 3: Split + FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs_split = splitter.split_documents(docs)

modelo_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vetores = FAISS.from_documents(docs_split, modelo_embed)
vetores.save_local("index_uibutton")

In [ ]:
# Etapa 4: Carregar FAISS + Tradutor + LLM
from transformers import pipeline
import torch

translator = pipeline("translation_en_to_pt", model="Helsinki-NLP/opus-mt-tc-big-en-pt", device=0 if torch.cuda.is_available() else -1)

from transformers import T5Tokenizer, T5ForConditionalGeneration
from langchain_huggingface import HuggingFacePipeline

model = T5ForConditionalGeneration.from_pretrained("unicamp-dl/ptt5-base-portuguese-vocab")
tokenizer = T5Tokenizer.from_pretrained("unicamp-dl/ptt5-base-portuguese-vocab", legacy=False)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
llm = HuggingFacePipeline(pipeline=pipe)

In [11]:
# Etapa 5: Criar retriever e QA Chain
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

index = FAISS.load_local("index_uibutton", modelo_embed, allow_dangerous_deserialization=True)
retriever = index.as_retriever(search_kwargs={"k": 3})

prompt_template = PromptTemplate(
    template="""Você é um especialista em UIKit.
Use somente o contexto fornecido para responder à pergunta.
Se não encontrar a resposta, diga "Não encontrei na documentação".

Contexto:
{context}

Pergunta:
{question}

Resposta:""",
    input_variables=["context", "question"]
)

import time

def traduzir_docs_para_contexto(docs):
    textos = [doc.page_content for doc in docs]
    inicio = time.time()
    traduzidos = translator(textos, max_length=1000)
    print(f"⏱️ Tradução: {time.time() - inicio:.2f}s")
    return "\n\n".join([t['translation_text'] for t in traduzidos])

qa_chain = (
    {"context": retriever | traduzir_docs_para_contexto, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [15]:
# Etapa 6: Função de pergunta interativa

def responder_uibutton(pergunta):
    try:
        resposta = qa_chain.invoke(pergunta).strip()

        # Remove instruções repetidas
        if "Você é um especialista em UIKit" in resposta:
            resposta = resposta.split("Contexto:")[-1].strip()

        indice_contexto = resposta.lower().rfind("contexto:")
        if indice_contexto != -1:
          resposta = resposta[len("Contexto:"):].strip()

        # Remove prefixo redundante
        if resposta.lower().startswith("resposta:"):
            resposta = resposta.strip()

        return resposta.strip()
    except Exception as e:
        return f"❌ Erro: {e}"

In [ ]:
from IPython.display import display, Markdown
import ipywidgets as widgets

entrada = widgets.Text(
    value='',
    placeholder='Digite sua pergunta sobre UIButton...',
    description='Pergunta:',
    layout=widgets.Layout(width='100%')
)

botao = widgets.Button(description="Enviar")
saida = widgets.Output()

def ao_enviar(_):
    pergunta = entrada.value.strip()
    if pergunta:
        with saida:
            saida.clear_output()
            print(f"👤 Você: {pergunta}")
            resposta = responder_uibutton(pergunta)
            display(Markdown("🤖 **Resposta:**\n\n" + resposta.replace("\n", "  \n")))
        entrada.value = ''

botao.on_click(ao_enviar)
display(widgets.VBox([entrada, botao, saida]))